## Modelling

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from pyspark.ml.regression import GeneralizedLinearRegression, RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [ ]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1 modelling")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "3g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

In [ ]:
# read training and test dataset
train_sdf = spark.read.parquet("../data/curated/new_train")
test_sdf = spark.read.parquet("../data/curated/new_test")

# check size of training/test set
print("Number of instances in traning data:", train_sdf.count())
print("Number of instances in test data:", test_sdf.count())

In [ ]:
# define which attributes are numerical versus categorical
label = "trip_time"
categorical_attr = ["day_of_week"]

# binary attribute does not require one-hot encoding
numerical_binary_attr = ["trip_miles", "Temperature (F)", "hour_of_day", "congestion_zone", 
                         "JFK_trip", "Newark_trip", "LaGuardia_trip", "Manhattan_trip"]

In [ ]:
# preprocess attributes and data in train and test sets
def preprocess_attribute(train_sdf, test_sdf):

    # apply one-hot encoding to categorical attributes
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_index") for c in categorical_attr]
    encoder = OneHotEncoder(
        inputCols=[indexer.getOutputCol() for indexer in indexers],
        outputCols=[
            "{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers]
    )

    # use VectorAssembler to assemble data together as a vector
    assembler = VectorAssembler(inputCols = encoder.getOutputCols() + numerical_binary_attr, outputCol = "features")

    pipeline = Pipeline(stages=indexers + [encoder, assembler])
    preprocess = pipeline.fit(train_sdf.dropna('any'))
    model_sdf = preprocess.transform(train_sdf)

    # apply log transformation on label 'trip_time' -> this is used to help compute RMSLE
    model_sdf = model_sdf.withColumn(label, F.log(model_sdf[label]))

    # display the features and targets for our model
    model_sdf.select('features').head(5), model_sdf.select('trip_time').head(5)

    # preprocess for predictions
    predict_sdf = preprocess.transform(test_sdf)
    predict_sdf = predict_sdf.withColumn(label, F.log(predict_sdf[label]))
    predict_sdf.show(1, vertical=True)

    return model_sdf, predict_sdf

model_sdf, predict_sdf = preprocess_attribute(train_sdf, test_sdf)

In [ ]:
# fit the model and apply to the test data
def train_predict(model, train, test):
    model = model.fit(train)
    predictions = model.transform(test)
    # predictions.show(1, vertical=True)
    return predictions

In [ ]:
# calculate RMSLE and R-squared for a given model
def evaluate(predictions):
    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(labelCol="trip_time", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print("Root Mean Squared Logarithmic Error (RMSLE) on test data = %g" % rmse)

    evaluator = RegressionEvaluator(labelCol="trip_time", predictionCol="prediction", metricName="r2")
    r2 = evaluator.evaluate(predictions)
    print("R Squared on test data = %g" % r2)

In [ ]:
# produce residual plot for trips involving JFK airport in sampled test data (sampling data for visualisation purpose)
def residual_plot(predictions, model_name):
    SAMPLE_SIZE = 0.001
    sample_prediction = predictions.select("trip_time", "prediction", "JFK_trip").sample(SAMPLE_SIZE, seed=0).toPandas()
    print("Number of instances in sample dataset:", sample_prediction.shape[0])

    # filter trips involving JFK airport
    jfk_trip_prediction = sample_prediction.loc[sample_prediction["JFK_trip"]==True]
    jfk_trip_prediction["residuals"] = jfk_trip_prediction["trip_time"] - jfk_trip_prediction["prediction"]
    print("Number of instances in JFK dataset:", jfk_trip_prediction.shape[0])
    
    sns.scatterplot(data=jfk_trip_prediction, x="prediction", y="residuals")
    plt.title("Residual Plot for Sampled Trips To or From JFK Airport", fontsize=13)
    plt.xlabel("Predicted Log transformed trip time", fontsize=12)
    plt.ylabel("Residuals", fontsize=12)
    plt.savefig(f"../plots/{model_name}_residual_plot.png", bbox_inches='tight')
    plt.show()

### Hyperparameter Tuning
Hyperparameter tuning is done through comparing evaluation results of models with different parameter values. Due to limited computing power and large training set, we cannot perform Cross Validation on all models. 
- Gamma GLM: models trained with different values of regularisation parameter (0.01, 0.1, 0.3, 0.5, 0.7, 0.9)
- Random Forest Regression: models trained with different number of trees (3, 5, 10, 12)

In [ ]:
# gamma GLM 
reg_param_list = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9]
for i in reg_param_list:
    print("regParam =", i)
    glr = GeneralizedLinearRegression(featuresCol='features', labelCol='trip_time', 
            family="gamma", link="log", maxIter=10, regParam=i)
    predictions = train_predict(glr, model_sdf, predict_sdf)
    evaluate(predictions)
    print("\n")

In [ ]:
# Random Forest Regression
num_tree_list = [3, 5, 10, 12]
for i in num_tree_list:
    print("numTrees =", i)
    rf = RandomForestRegressor(featuresCol='features', labelCol='trip_time', numTrees=i, maxDepth=5, seed=0)
    predictions = train_predict(rf, model_sdf, predict_sdf)
    evaluate(predictions)
    print("\n")

### Random Forest Regressor
Optimal model: numTrees=5

In [ ]:
# fit the training data with the optimal model (5 trees) found in hyperparameter tuning
rf = RandomForestRegressor(featuresCol='features', labelCol='trip_time', numTrees=5, maxDepth=5, seed=0)
predictions = train_predict(rf, model_sdf, predict_sdf)
evaluate(predictions)
residual_plot(predictions, "RF")

### Generalized Linear Regression
Optimal model: regParam=0.9

In [ ]:
# # fit the training data with the optimal model (regParam=0.9)
glr = GeneralizedLinearRegression(featuresCol='features', labelCol='trip_time', 
            family="gamma", link="log", maxIter=10, regParam=0.9)

predictions = train_predict(glr, model_sdf, predict_sdf)
evaluate(predictions)
residual_plot(predictions, "GLM")